In [1]:
## Introduction

## Data intro

### Importing Libraries

In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

## Loading & Inspecting Data

In [15]:
df = pd.read_csv("multi_app_reviews.csv")
print(df.shape)
print(df['app'].value_counts())
print(df['lang'].value_counts())
print(df['score'].value_counts())
df.head(10)

(9468, 4)
app
Easypaisa    2000
JazzCash     1664
Daraz        1287
OLX          1215
Careem       1159
FoodPanda    1089
Bykea        1054
Name: count, dtype: int64
lang
en    7000
ur    2468
Name: count, dtype: int64
score
5    5621
1    2686
4     461
3     374
2     326
Name: count, dtype: int64


,app,lang,content,score
0,Daraz,ur,Some things look big and the real thing is small.,3
1,Daraz,ur,The great app for shopping and fulfil your nee...,5
2,Daraz,ur,mindblowing app,5
3,Daraz,ur,nice,5
4,Daraz,ur,کچھ چیزیں منگوائے ہیں دیکھتے ہیں دراز کتننا سچ...,5
5,Daraz,ur,good,5
6,Daraz,ur,نہیں ضرورت جی ختم کر دو,1
7,Daraz,ur,اچھی ایپ ہے,5
8,Daraz,ur,ماشاءاللہ زبردست ھم نے سنا ہے کہ دراز بیسٹ ہے ...,4
9,Daraz,ur,best online store,5


## Basic Cleaning

- Remove duplicates & very short reviews (len(text) < 3).

- Normalize text (lowercase, strip spaces).

- Drop empty reviews.

In [16]:
df.drop_duplicates(subset="content", inplace=True)
df = df[df['content'].str.len() > 3]
df['content'] = df['content'].str.lower().str.strip()

In [17]:
df.head()

,app,lang,content,score
0,Daraz,ur,some things look big and the real thing is small.,3
1,Daraz,ur,the great app for shopping and fulfil your nee...,5
2,Daraz,ur,mindblowing app,5
3,Daraz,ur,nice,5
4,Daraz,ur,کچھ چیزیں منگوائے ہیں دیکھتے ہیں دراز کتننا سچ...,5


In [18]:
df.shape

(6644, 4)

In [19]:
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0  # for consistent results

def detect_lang(text):
    try:
        return detect(text)
    except:
        return "unknown"

In [20]:
df['detected_lang'] = df['content'].apply(detect_lang)

In [21]:
roman_urdu_words = ['acha','boht','nahi','masla','bhai','fraud','kam','apka','ghalat','theek']

def is_roman_urdu(text):
    return any(word in text for word in roman_urdu_words)

df = df[~((df['detected_lang'] == 'en') & (~df['content'].apply(is_roman_urdu)))]

In [22]:
print(df.shape)
print(df['app'].value_counts())
print(df['lang'].value_counts())
print(df['score'].value_counts())
df.head(10)

(3054, 5)
app
Easypaisa    762
JazzCash     558
Daraz        482
OLX          353
FoodPanda    316
Careem       310
Bykea        273
Name: count, dtype: int64
lang
en    1761
ur    1293
Name: count, dtype: int64
score
5    1878
1     811
4     133
3     122
2     110
Name: count, dtype: int64


,app,lang,content,score,detected_lang
1,Daraz,ur,the great app for shopping and fulfil your nee...,5,en
3,Daraz,ur,nice,5,pl
4,Daraz,ur,کچھ چیزیں منگوائے ہیں دیکھتے ہیں دراز کتننا سچ...,5,ur
5,Daraz,ur,good,5,so
6,Daraz,ur,نہیں ضرورت جی ختم کر دو,1,ur
7,Daraz,ur,اچھی ایپ ہے,5,ur
8,Daraz,ur,ماشاءاللہ زبردست ھم نے سنا ہے کہ دراز بیسٹ ہے ...,4,ur
9,Daraz,ur,best online store,5,no
10,Daraz,ur,bohot achha app hai time pe delivery deta hai ...,5,id
11,Daraz,ur,fair,1,cy


In [23]:
# Function to assign sentiment from score
def label_sentiment(score):
    if score >= 4:
        return "positive"
    elif score == 3:
        return "neutral"
    else:
        return "negative"

In [24]:
# Apply labeling
df['sentiment'] = df['score'].apply(label_sentiment)

In [25]:
# Check distribution
print(df['sentiment'].value_counts())

sentiment
positive    2011
negative     921
neutral      122
Name: count, dtype: int64


In [26]:
# Save dataset
df.to_csv("star_labeled_reviews.csv", index=False)
print("Labeled dataset saved with shape:", df.shape)

Labeled dataset saved with shape: (3054, 6)


In [38]:
df.head()

,app,lang,content,score,detected_lang,sentiment
1,Daraz,ur,the great app for shopping and fulfil your nee...,5,en,positive
3,Daraz,ur,nice,5,pl,positive
4,Daraz,ur,کچھ چیزیں منگوائے ہیں دیکھتے ہیں دراز کتننا سچ...,5,ur,positive
5,Daraz,ur,good,5,so,positive
6,Daraz,ur,نہیں ضرورت جی ختم کر دو,1,ur,negative


## Dropping Less then 3 Word Words

In [39]:
def is_english_short(text, lang):
    words = text.split()
    if lang != "ur" and len(words) <= 3:
        return True
    return False
df = df[~df.apply(lambda row: is_english_short(row['content'], row['lang']), axis=1)]

In [40]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.0 MB/s eta 0:00:00


In [41]:
from google_play_scraper import Sort, reviews

In [56]:
from youtube_comment_downloader import *

In [57]:
# Initialize the downloader
downloader = YoutubeCommentDownloader()

# Download comments
video_url = "https://www.youtube.com/watch?v=Qtydsn3wYO4"
comments = downloader.get_comments_from_url(video_url)

# Convert to DataFrame
df = pd.DataFrame(comments)

# Save to CSV
df.to_csv("vlog_comments.csv", index=False)
print(f"Downloaded {len(df)} comments.")

Downloaded 13734 comments.


In [69]:
df = pd.read_csv("vlog_comments.csv")

# --- Step 1: Filter Urdu + Roman Urdu ---
def is_urdu_or_roman(text):
    try:
        lang = detect(text)
        return lang == 'ur'
    except:
        return False

df['is_urdu'] = df['text'].apply(is_urdu_or_roman)
df = df[df['is_urdu']].copy()
print(f"Filtered Urdu/Roman Urdu comments: {len(df)}")

Filtered Urdu/Roman Urdu comments: 147


In [ ]:
# Define Keywords with Spelling Variations
positive_keywords = [
    "achha","achha","achaa","best","mast","shandar","kamal",
    "zabardast","khoobsurat","behtareen","bohat acha","bohot acha"
]

negative_keywords = [
    "kharab","kharaab","weak","pareshan","bura","ganda","bekar",
    "maanhoos","gussa","dukhi","tang"
]

neutral_keywords = [
    "theek","thik","thek","normal","sahi","halka","zaroori",
    "waqt","din","jaisa"
]
# Lowercase all texts for matching
df['text_lower'] = df['text'].str.lower()

In [ ]:
# Convert votes to numeric
df['votes'] = pd.to_numeric(df['votes'], errors='coerce').fillna(0).astype(int)

# Convert heart to boolean
df['heart'] = df['heart'].astype(str).str.lower().isin(['true','1','yes'])

In [ ]:
# Keyword-based labeling
def keyword_sentiment(row):
    text = row['text_lower']
    # Hearted or high votes = positive
    if row.get('heart', False) or row.get('votes', 0) > 10:
        return "positive"
    # Negative keywords
    elif any(k in text for k in negative_keywords):
        return "negative"
    # Positive keywords
    elif any(k in text for k in positive_keywords):
        return "positive"
    # Neutral keywords
    elif any(k in text for k in neutral_keywords):
        return "neutral"
    else:
        return "neutral"  # default if none matches

In [73]:
df['sentiment'] = df.apply(keyword_sentiment, axis=1)

In [75]:
df.sentiment.value_counts()

,count
sentiment,
neutral,139
positive,8


In [76]:
df.shape

(147, 14)

In [77]:
df.to_csv('labeled_comments_data.csv')

## **combining Both files**

In [78]:
df_keyword = pd.read_csv("labeled_comments_data.csv")

# Keep only text + sentiment
df_keyword = df_keyword[['text', 'sentiment']].copy()
print("Keyword-labeled data shape:", df_keyword.shape)

Keyword-labeled data shape: (147, 2)


In [81]:
df_star = pd.read_csv("star_labeled_reviews.csv")
df_star['text'] = df_star['content']
# Ensure columns are named the same
df_star = df_star[['text', 'sentiment']].copy()
print("Star-labeled data shape:", df_star.shape)

Star-labeled data shape: (3054, 2)


In [ ]:
# Combine both datasets
df_combined = pd.concat([df_star, df_keyword], ignore_index=True)
print("Combined dataset shape:", df_combined.shape)

Combined dataset shape: (3201, 2)


In [ ]:
# Shuffle the dataset 
df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

# Save combined dataset 
df_combined.to_csv("combined_labeled_urdu_data.csv", index=False)
print("Combined labeled dataset saved.")

Combined labeled dataset saved.


In [84]:
df_combined.head()

,text,sentiment
0,excllent,positive
1,never trying,positive
2,bhai sahab ek number app hai mara tub bhi use ...,positive
3,السلام علیکم گزارش ہے کہ آگے چل کر کوئ چارجز ن...,positive
4,.🥹🥹🥹,positive


## Synthetic Data Using GPT

In [87]:
data = [
("JazzCash ka interface simple hai aur bill payment instantly ho jata hai", "positive"),
("Easypaisa se paisay transfer karna bohot fast hai, kabhi issue nahi aaya", "positive"),
("Bykea rider time pe aa gaya aur ride comfortable thi", "positive"),
("Careem captain ne polite behave kiya aur car clean thi", "positive"),
("FoodPanda se khana garam aur time par deliver hua, bohot acha experience", "positive"),
("Daraz se order kia mobile cover same as picture aaya, satisfied hoon", "positive"),
("OLX pe achi deals milti hain, is bar sofa bohot acha nikla", "positive"),
("Ye YouTube vlogger honest review deta hai, videos helpful hain", "positive"),
("Bykea delivery service fast aur cheap hai, bohot pasand aya", "positive"),
("Easypaisa app ka new update smooth hai aur loading time kam ho gaya", "positive"),
("FoodPanda ka customer support bohat cooperative hai, issue solve kar dia jaldi", "positive"),
("Careem ne mujhe late night safe drop off dia, trust barh gaya", "positive"),
("Daraz ki 11.11 sale mein genuine products milay, acha laga", "positive"),
("OLX par seller reliable tha, cheez same mili jo ad mein thi", "positive"),
("JazzCash se easy load lena 2 second ka kaam hai, bohat easy hai", "positive"),
("FoodPanda ki tracking feature helpful hai, order ka status clearly show hota hai", "positive"),
("Careem wallet recharge karna simple hai aur instant reflect hota hai", "positive"),
("Bykea ride ka rate reasonable tha, aur rider friendly bhi", "positive"),
("Daraz app ka search option improve hua hai, results relevant miltay hain", "positive"),
("Easypaisa QR payment feature bohot convenient hai, dukaan pe easy pay ho jata hai", "positive"),
("FoodPanda par naye restaurants explore karne ka option acha lagta hai", "positive"),
("JazzCash debit card online shopping ke liye bohot useful hai", "positive"),
("Careem captain ne long route nahi lia, fair accurate tha", "positive"),
("Daraz mall products original milay, trust build ho raha hai", "positive"),
("OLX pe price negotiate karna easy tha aur deal fair hui", "positive"),
("YouTube review ne mujhe laptop choose karne mein help di, honest tha", "positive"),
("Bykea ka referral system acha hai, free ride mil gayi", "positive"),
("Easypaisa se electricity bill pay karna hassle free hai", "positive"),
("FoodPanda discount codes helpful hain, order cheap pad gaya", "positive"),
("JazzCash se university fees pay ki easily, koi problem nahi hui", "positive"),
("Careem ki car bohot clean aur AC perfect chal raha tha", "positive"),
("Daraz ka order packing safe thi, product damage free mila", "positive"),
("OLX seller ne fast response dia aur meeting easy rahi", "positive"),
("FoodPanda ka rider punctual tha aur polite bhi", "positive"),
("Easypaisa mini app features helpful hain, recharge aur shopping easy hogi", "positive"),
("Bykea parcels time par pohanch gaye, reliable lagta hai service", "positive"),
("Careem ne mujhe points diye jo discount mein convert hue, acha system hai", "positive"),
("JazzCash ka biometric verification fast tha, line mein wait nahi karna pada", "positive"),
("Daraz par flash sale se acha mobile sasta mil gaya", "positive"),
("FoodPanda app bohot user friendly hai, sab kuch easily mil jata hai", "positive"),
("FoodPanda se order kia tha lekin khana thanda aur late deliver hua", "negative"),
("JazzCash se paisa send kia tha par recipient tak nahi pohancha", "negative"),
("Careem captain ne ride cancel kar di last moment pe, time waste hua", "negative"),
("Daraz se shoes order kiye thay lekin quality bohot low nikli", "negative"),
("Easypaisa app bar bar crash ho rahi hai, bilkul kaam nahi kar rahi", "negative"),
("Bykea rider rude tha aur route galat le gaya", "negative"),
("OLX seller ne fake product dikhaya aur price bhi high tha", "negative"),
("YouTube vlogger sirf sponsored cheezein promote karta hai, trust nahi hota", "negative"),
("JazzCash ka customer service call uthata hi nahi, bohot irritating", "negative"),
("Careem ka fare estimate aur final bill mein bohot difference tha", "negative"),
("Daraz order cancel kar deta hai bina reason diye, frustration hoti hai", "negative"),
("FoodPanda app par tracking bilkul accurate nahi hoti, rider ka pata nahi chalta", "negative"),
("Easypaisa withdrawal par hidden charges lagte hain, bilkul unfair hai", "negative"),
("Bykea delivery bohot late pohanchi aur parcel damaged tha", "negative"),
("OLX pe scams bohot zyada hain, safe nahi lagta", "negative"),
("YouTube review bilkul biased tha, sach nahi lagta", "negative"),
("JazzCash ka app interface complicated hai, easily samajh nahi aata", "negative"),
("Daraz ki packing poor thi aur product thoda broken nikla", "negative"),
("Careem rider ne AC off kar dia aur zyada paisay maange", "negative"),
("FoodPanda se wrong item deliver hua aur support ne help nahi ki", "negative"),
("Easypaisa se bill pay kia lekin system pe update hi nahi hua", "negative"),
("Bykea app ka location system glitch karta hai, rider galat jagah chala jata hai", "negative"),
("JazzCash wallet balance deduct ho gaya lekin transaction failed aayi", "negative"),
("Daraz pe customer service slow aur useless hoti hai", "negative"),
("OLX pe fake ads bohot hain aur verification proper nahi", "negative"),
("FoodPanda ke discount codes kaam hi nahi karte, waste of time", "negative"),
("Careem support team response bohot late karti hai", "negative"),
("Bykea rider ne parcel dus din baad deliver kia, unacceptable", "negative"),
("Easypaisa app update ke baad aur slow ho gayi hai", "negative"),
("JazzCash ka helpline number busy rehta hai, contact hi nahi hota", "negative"),
("Daraz return policy follow karne ke bawajood refund nahi mila", "negative"),
("FoodPanda ka rider customer se baat karte hue rude tha", "negative"),
("Careem captain ne extra charges demand kar liye bina reason ke", "negative"),
("Easypaisa account verification process bohot confusing hai", "negative"),
("Bykea app freeze ho jati hai mid-ride, bohot annoying hai", "negative"),
("OLX par fraudsters active hain, safe nahi feel hota", "negative"),
("YouTube vlogger overhype karta hai products ko, genuine review nahi lagta", "negative"),
("Daraz pe delivery bohot slow hoti hai, weeks lag jate hain", "negative"),
("FoodPanda support ne complaint ignore kar di, resolve hi nahi hua", "negative"),
("JazzCash aur Easypaisa dono convenient hain, bas charges compare karne chahiye", "neutral"),
("Careem ka fare thoda zyada lagta hai lekin ride service theek hai", "neutral"),
("Daraz par products ka mix hota hai, kabhi acha kabhi average", "neutral"),
("FoodPanda se delivery kabhi fast hoti hai kabhi late, depends on area", "neutral"),
("OLX pe cheez milti bhi hai aur kabhi waste of time bhi ho jata hai", "neutral"),
("Bykea ka service theek hai, lekin kabhi delay bhi ho jata hai", "neutral"),
("YouTube pe kuch vloggers genuine lagte hain aur kuch sponsored", "neutral"),
("Easypaisa aur JazzCash dono ke apps barabar chal rahe hain", "neutral"),
("Daraz sales mein price kam hota hai lekin stock jaldi khatam ho jata hai", "neutral"),
("Careem aur InDrive dono apps ka apna apna benefit hai", "neutral"),
("FoodPanda ka discount kabhi work karta hai kabhi nahi", "neutral"),
("JazzCash ka interface average hai, not too good not too bad", "neutral"),
("Bykea ke riders kabhi punctual hote hain aur kabhi late", "neutral"),
("Easypaisa ka helpline kabhi helpful hota hai kabhi busy hota hai", "neutral"),
("OLX par deals karna safe tab lagta hai jab seller reliable ho", "neutral"),
("Daraz ke reviews hamesha accurate nahi hote, mix experience hota hai", "neutral"),
("Careem captain kabhi friendly milta hai aur kabhi normal", "neutral"),
("FoodPanda ki service city to city differ karti hai", "neutral"),
("YouTube reviews kabhi genuine lagte hain aur kabhi overhyped", "neutral"),
("JazzCash aur Easypaisa dono ka transaction speed similar hai", "neutral"),
("Bykea ka fare reasonable hota hai lekin traffic delay ho jata hai", "neutral"),
("Daraz par product variety bohot hai lekin quality har bar same nahi hoti", "neutral"),
("Careem app theek chalti hai, kabhi kabhi hang bhi karti hai", "neutral"),
("Easypaisa se recharge smooth hai, kabhi kabhi server error aa jata hai", "neutral"),
("OLX use karna easy hai lekin har ad pe trust nahi hota", "neutral"),
("FoodPanda app simple hai, lekin kabhi glitch bhi karti hai", "neutral"),
("YouTube vlogs interesting hote hain lekin har time informative nahi hote", "neutral"),
("JazzCash app user friendly hai, lekin updates kabhi slow kar dete hain", "neutral"),
("Daraz return policy theek hai, lekin time consuming hoti hai", "neutral"),
("Careem aur Bykea dono ka apna market hai, depend karta hai user ki need pe", "neutral"),
("FoodPanda ka rider kabhi fast milta hai kabhi wait karna padta hai", "neutral"),
("Easypaisa aur JazzCash charges zyada different nahi lagte", "neutral"),
("OLX pe buyer aur seller dono par depend karta hai experience", "neutral"),
("Bykea se short rides theek lagti hain, long rides costly hoti hain", "neutral"),
("YouTube product reviews har bar trust worthy nahi lagte", "neutral"),
("Daraz pe delivery experience area ke hisaab se change hota hai", "neutral"),
("Careem ka AC kabhi theek chalta hai kabhi weak hota hai", "neutral"),
("FoodPanda app ka UI simple hai, lekin kuch features missing hain", "neutral")
]

df = pd.DataFrame(data, columns=["text", "sentiment"])
df.to_csv("urdu_roman_synthetic_initial.csv", index=False, encoding="utf-8")

In [89]:
import pandas as pd

data = [
("JazzCash ka interface simple hai aur bill payment instantly ho jata hai", "positive"),
("Easypaisa se paisay transfer karna bohot fast hai, kabhi issue nahi aaya", "positive"),
("Bykea rider time pe aa gaya aur ride comfortable thi", "positive"),
("Careem captain ne polite behave kiya aur car clean thi", "positive"),
("FoodPanda se khana garam aur time par deliver hua, bohot acha experience", "positive"),
("Daraz se order kia mobile cover same as picture aaya, satisfied hoon", "positive"),
("OLX pe achi deals milti hain, is bar sofa bohot acha nikla", "positive"),
("Ye YouTube vlogger honest review deta hai, videos helpful hain", "positive"),
("Bykea delivery service fast aur cheap hai, bohot pasand aya", "positive"),
("Easypaisa app ka new update smooth hai aur loading time kam ho gaya", "positive"),
("FoodPanda ka customer support bohat cooperative hai, issue solve kar dia jaldi", "positive"),
("Careem ne mujhe late night safe drop off dia, trust barh gaya", "positive"),
("Daraz ki 11.11 sale mein genuine products milay, acha laga", "positive"),
("OLX par seller reliable tha, cheez same mili jo ad mein thi", "positive"),
("JazzCash se easy load lena 2 second ka kaam hai, bohat easy hai", "positive"),
("FoodPanda ki tracking feature helpful hai, order ka status clearly show hota hai", "positive"),
("Careem wallet recharge karna simple hai aur instant reflect hota hai", "positive"),
("Bykea ride ka rate reasonable tha, aur rider friendly bhi", "positive"),
("Daraz app ka search option improve hua hai, results relevant miltay hain", "positive"),
("Easypaisa QR payment feature bohot convenient hai, dukaan pe easy pay ho jata hai", "positive"),
("FoodPanda par naye restaurants explore karne ka option acha lagta hai", "positive"),
("JazzCash debit card online shopping ke liye bohot useful hai", "positive"),
("Careem captain ne long route nahi lia, fair accurate tha", "positive"),
("Daraz mall products original milay, trust build ho raha hai", "positive"),
("OLX pe price negotiate karna easy tha aur deal fair hui", "positive"),
("YouTube review ne mujhe laptop choose karne mein help di, honest tha", "positive"),
("Bykea ka referral system acha hai, free ride mil gayi", "positive"),
("Easypaisa se electricity bill pay karna hassle free hai", "positive"),
("FoodPanda discount codes helpful hain, order cheap pad gaya", "positive"),
("JazzCash se university fees pay ki easily, koi problem nahi hui", "positive"),
("Careem ki car bohot clean aur AC perfect chal raha tha", "positive"),
("Daraz ka order packing safe thi, product damage free mila", "positive"),
("OLX seller ne fast response dia aur meeting easy rahi", "positive"),
("FoodPanda ka rider punctual tha aur polite bhi", "positive"),
("Easypaisa mini app features helpful hain, recharge aur shopping easy hogi", "positive"),
("Bykea parcels time par pohanch gaye, reliable lagta hai service", "positive"),
("Careem ne mujhe points diye jo discount mein convert hue, acha system hai", "positive"),
("JazzCash ka biometric verification fast tha, line mein wait nahi karna pada", "positive"),
("Daraz par flash sale se acha mobile sasta mil gaya", "positive"),
("FoodPanda app bohot user friendly hai, sab kuch easily mil jata hai", "positive"),
("FoodPanda se order kia tha lekin khana thanda aur late deliver hua", "negative"),
("JazzCash se paisa send kia tha par recipient tak nahi pohancha", "negative"),
("Careem captain ne ride cancel kar di last moment pe, time waste hua", "negative"),
("Daraz se shoes order kiye thay lekin quality bohot low nikli", "negative"),
("Easypaisa app bar bar crash ho rahi hai, bilkul kaam nahi kar rahi", "negative"),
("Bykea rider rude tha aur route galat le gaya", "negative"),
("OLX seller ne fake product dikhaya aur price bhi high tha", "negative"),
("YouTube vlogger sirf sponsored cheezein promote karta hai, trust nahi hota", "negative"),
("JazzCash ka customer service call uthata hi nahi, bohot irritating", "negative"),
("Careem ka fare estimate aur final bill mein bohot difference tha", "negative"),
("Daraz order cancel kar deta hai bina reason diye, frustration hoti hai", "negative"),
("FoodPanda app par tracking bilkul accurate nahi hoti, rider ka pata nahi chalta", "negative"),
("Easypaisa withdrawal par hidden charges lagte hain, bilkul unfair hai", "negative"),
("Bykea delivery bohot late pohanchi aur parcel damaged tha", "negative"),
("OLX pe scams bohot zyada hain, safe nahi lagta", "negative"),
("YouTube review bilkul biased tha, sach nahi lagta", "negative"),
("JazzCash ka app interface complicated hai, easily samajh nahi aata", "negative"),
("Daraz ki packing poor thi aur product thoda broken nikla", "negative"),
("Careem rider ne AC off kar dia aur zyada paisay maange", "negative"),
("FoodPanda se wrong item deliver hua aur support ne help nahi ki", "negative"),
("Easypaisa se bill pay kia lekin system pe update hi nahi hua", "negative"),
("Bykea app ka location system glitch karta hai, rider galat jagah chala jata hai", "negative"),
("JazzCash wallet balance deduct ho gaya lekin transaction failed aayi", "negative"),
("Daraz pe customer service slow aur useless hoti hai", "negative"),
("OLX pe fake ads bohot hain aur verification proper nahi", "negative"),
("FoodPanda ke discount codes kaam hi nahi karte, waste of time", "negative"),
("Careem support team response bohot late karti hai", "negative"),
("Bykea rider ne parcel dus din baad deliver kia, unacceptable", "negative"),
("Easypaisa app update ke baad aur slow ho gayi hai", "negative"),
("JazzCash ka helpline number busy rehta hai, contact hi nahi hota", "negative"),
("Daraz return policy follow karne ke bawajood refund nahi mila", "negative"),
("FoodPanda ka rider customer se baat karte hue rude tha", "negative"),
("Careem captain ne extra charges demand kar liye bina reason ke", "negative"),
("Easypaisa account verification process bohot confusing hai", "negative"),
("Bykea app freeze ho jati hai mid-ride, bohot annoying hai", "negative"),
("OLX par fraudsters active hain, safe nahi feel hota", "negative"),
("YouTube vlogger overhype karta hai products ko, genuine review nahi lagta", "negative"),
("Daraz pe delivery bohot slow hoti hai, weeks lag jate hain", "negative"),
("FoodPanda support ne complaint ignore kar di, resolve hi nahi hua", "negative"),
("JazzCash aur Easypaisa dono convenient hain, bas charges compare karne chahiye", "neutral"),
("Careem ka fare thoda zyada lagta hai lekin ride service theek hai", "neutral"),
("Daraz par products ka mix hota hai, kabhi acha kabhi average", "neutral"),
("FoodPanda se delivery kabhi fast hoti hai kabhi late, depends on area", "neutral"),
("OLX pe cheez milti bhi hai aur kabhi waste of time bhi ho jata hai", "neutral"),
("Bykea ka service theek hai, lekin kabhi delay bhi ho jata hai", "neutral"),
("YouTube pe kuch vloggers genuine lagte hain aur kuch sponsored", "neutral"),
("Easypaisa aur JazzCash dono ke apps barabar chal rahe hain", "neutral"),
("Daraz sales mein price kam hota hai lekin stock jaldi khatam ho jata hai", "neutral"),
("Careem aur InDrive dono apps ka apna apna benefit hai", "neutral"),
("FoodPanda ka discount kabhi work karta hai kabhi nahi", "neutral"),
("JazzCash ka interface average hai, not too good not too bad", "neutral"),
("Bykea ke riders kabhi punctual hote hain aur kabhi late", "neutral"),
("Easypaisa ka helpline kabhi helpful hota hai kabhi busy hota hai", "neutral"),
("OLX par deals karna safe tab lagta hai jab seller reliable ho", "neutral"),
("Daraz ke reviews hamesha accurate nahi hote, mix experience hota hai", "neutral"),
("Careem captain kabhi friendly milta hai aur kabhi normal", "neutral"),
("FoodPanda ki service city to city differ karti hai", "neutral"),
("YouTube reviews kabhi genuine lagte hain aur kabhi overhyped", "neutral"),
("JazzCash aur Easypaisa dono ka transaction speed similar hai", "neutral"),
("Bykea ka fare reasonable hota hai lekin traffic delay ho jata hai", "neutral"),
("Daraz par product variety bohot hai lekin quality har bar same nahi hoti", "neutral"),
("Careem app theek chalti hai, kabhi kabhi hang bhi karti hai", "neutral"),
("Easypaisa se recharge smooth hai, kabhi kabhi server error aa jata hai", "neutral"),
("OLX use karna easy hai lekin har ad pe trust nahi hota", "neutral"),
("FoodPanda app simple hai, lekin kabhi glitch bhi karti hai", "neutral"),
("YouTube vlogs interesting hote hain lekin har time informative nahi hote", "neutral"),
("JazzCash app user friendly hai, lekin updates kabhi slow kar dete hain", "neutral"),
("Daraz return policy theek hai, lekin time consuming hoti hai", "neutral"),
("Careem aur Bykea dono ka apna market hai, depend karta hai user ki need pe", "neutral"),
("FoodPanda ka rider kabhi fast milta hai kabhi wait karna padta hai", "neutral"),
("Easypaisa aur JazzCash charges zyada different nahi lagte", "neutral"),
("OLX pe buyer aur seller dono par depend karta hai experience", "neutral"),
("Bykea se short rides theek lagti hain, long rides costly hoti hain", "neutral"),
("YouTube product reviews har bar trust worthy nahi lagte", "neutral"),
("Daraz pe delivery experience area ke hisaab se change hota hai", "neutral"),
("Careem ka AC kabhi theek chalta hai kabhi weak hota hai", "neutral"),
("FoodPanda app ka UI simple hai, lekin kuch features missing hain", "neutral"),
("JazzCash se mobile load karna bilkul easy hai, ek do click mein kaam ho jata hai", "positive"),
("Easypaisa ka cashback offer acha laga, groceries pe bachat ho gayi", "positive"),
("Bykea ki ride short distance ke liye perfect hai, sasti aur fast", "positive"),
("Careem ne mujhe airport se pick kia aur driver punctual tha, zabardast service", "positive"),
("FoodPanda par restaurant variety bohot hai, naye cuisines try karne ka maza ata hai", "positive"),
("Daraz se headphones order kiye aur sound quality expected se better thi", "positive"),
("OLX par ek old bike khareedi aur condition waqai achi thi, seller ne sach bola", "positive"),
("Ye YouTube vlogger ka style casual tha lekin review bohot informative laga", "positive"),
("JazzCash ne electricity bill instantly clear kar dia, hassle free process", "positive"),
("Easypaisa se university fees pay krna easy tha aur receipt turant mil gayi", "positive"),
("Bykea rider ne polite tareeqe se baat ki aur time save ho gaya", "positive"),
("Careem rewards points ko redeem kar k discount lena bohot acha lagta hai", "positive"),
("FoodPanda ka app interface clean hai, use karna simple hai", "positive"),
("Daraz ka live tracking helpful tha, pata chalta raha parcel kidhar hai", "positive"),
("OLX seller cooperative tha aur price reasonable rakha", "positive"),
("JazzCash debit card ka design stylish bhi hai aur kaam bhi smooth karta hai", "positive"),
("Easypaisa ne new update ke baad login process fast kar diya", "positive"),
("Bykea ki delivery reliable hai, office documents safe pohanch gaye", "positive"),
("Careem captain professional tha aur seat belt remind bhi karwaya", "positive"),
("FoodPanda ke notifications timely hain, kabhi order miss nahi hota", "positive"),
("Daraz mall products pe trust hai, ab sirf wahi prefer karta hoon", "positive"),
("YouTube ka cooking channel genuine lagta hai, recipes simple aur tasty banti hain", "positive"),
("JazzCash ka customer support iss dafa helpful tha, issue 5 minute mein solve ho gaya", "positive"),
("Easypaisa wallet ka balance check karna instant hai, pasand aya", "positive"),
("Bykea se parcel bhejna economical hai, courier ke muqable mein sasta", "positive"),
("Careem ne exact pickup location pe aana easy kar dia, GPS accurate tha", "positive"),
("FoodPanda par kuch restaurants free delivery dete hain, bohot acha lagta hai", "positive"),
("Daraz app pe filter option useful hai, jaldi required cheez mil jati hai", "positive"),
("OLX par ek laptop liya aur condition original packing jaisi thi", "positive"),
("JazzCash app ka dark mode feature acha lagta hai", "positive"),
("Easypaisa ke seasonal offers se savings hoti hain", "positive"),
("Bykea ka cancellation fee low hai, isi liye prefer karta hoon", "positive"),
("Careem ki car fragrance fresh thi, trip mazedaar tha", "positive"),
("FoodPanda ke rider friendly hote hain, polite baat karte hain", "positive"),
("Daraz se gaming mouse order kia aur packaging professional thi", "positive"),
("YouTube ke reviews genuine aur bina overhype ke lagte hain, acha experience", "positive"),
("OLX pe customer support fast response deta hai, issue resolve ho gaya", "positive"),
("Easypaisa app ka design clean aur modern hai, easy to use", "positive"),
("Bykea ka interface user friendly hai, easily ride book ho jati hai", "positive"),
("Careem ne estimated time sahi diya aur ride bilkul on time thi", "positive"),
("FoodPanda ka order galat aaya, biryani ki jagah pulao bhej dia", "negative"),
("JazzCash app login bar bar fail hota hai, irritating hai", "negative"),
("Easypaisa se transfer kia paisa pending mein atak gaya, bohot bura experience", "negative"),
("Bykea rider ne route unnecessarily lamba le lia, fare zyada aa gaya", "negative"),
("Careem captain ka behaviour rough tha, uncomfortable ride hui", "negative"),
("Daraz se mobile cover order kia lekin size fit hi nahi tha", "negative"),
("OLX pe ek ad fake nikla, seller ne respond karna band kar dia", "negative"),
("YouTube vlogger sirf free products dikhata hai, sach nahi bolta", "negative"),
("JazzCash balance deduct ho gaya lekin recharge nahi hua", "negative"),
("Easypaisa app update ke baad aur heavy ho gayi hai, slow chalti hai", "negative"),
("Bykea delivery late thi aur parcel half torn tha", "negative"),
("Careem app hang ho gayi trip ke darmiyan, bohot problem hua", "negative"),
("FoodPanda ka discount code apply hi nahi hota, waste of time", "negative"),
("Daraz ki packaging weak thi aur item scratch hua aaya", "negative"),
("OLX pe fraudster se pala para, advance paisay le kar bhaag gaya", "negative"),
("YouTube channel sirf clickbait titles use karta hai, content weak hai", "negative"),
("JazzCash se fees pay ki thi par system pe reflect hi nahi hua", "negative"),
("Easypaisa se bill pay kia lekin duplicate charges lag gaye", "negative"),
("Bykea rider bohot late aya aur excuse banata raha", "negative"),
("Careem ka fare har dafa unpredictable hota hai, reliable nahi", "negative"),
("FoodPanda rider rude tone mein baat kar raha tha", "negative"),
("Daraz return process lengthy aur frustrating hai", "negative"),
("OLX pe buyer ne fake note de dia, bohot risky hai", "negative"),
("YouTube review biased tha, sponsor ki tareef hi karta raha", "negative"),
("JazzCash ke hidden charges bohot zyada hain, bilkul unfair", "negative"),
("Easypaisa ki customer service se baat karna mushkil hai", "negative"),
("Bykea app bar bar crash hoti hai, ride book hi nahi hoti", "negative"),
("Careem ne extra surcharges add kar diye without reason", "negative"),
("FoodPanda support ne complaint ignore kar di, bilkul help nahi ki", "negative"),
("Daraz delivery time promise se double lag gaya", "negative"),
("OLX par verification system weak hai, fake ads easily post ho jate hain", "negative"),
("JazzCash account block kar dia bina prior notice, inconvenience hua", "negative"),
("Easypaisa ne OTP send hi nahi kia, login impossible ho gaya", "negative"),
("Bykea ne parcel galat address pe deliver kar dia", "negative"),
("Careem captain ne ride cancel kar di 3 dafa, time waste", "negative"),
("FoodPanda par khanay ki quality down ho rahi hai", "negative"),
("Daraz se shoes order kiye aur dono alag size ke aa gaye", "negative"),
("YouTube vlogger overacting karta hai, review seriously nahi le sakte", "negative"),
("OLX seller ne original keh kar duplicate item dia", "negative"),
("JazzCash app ka layout confusing hai, easily navigate nahi hota", "negative"),
("JazzCash aur Easypaisa dono use karta hoon, ek hi din mein mixed results milte hain", "neutral"),
("Careem ka fare kabhi normal lagta hai kabhi high, location pe depend karta hai", "neutral"),
("Daraz ka order experience har bar alag hota hai, kabhi acha kabhi average", "neutral"),
("FoodPanda se kabhi fresh food milta hai aur kabhi thoda late aata hai", "neutral"),
("OLX pe deals karna risk bhi hai aur faida bhi mil sakta hai", "neutral"),
("Bykea ka ride service short trips pe theek lagta hai, long ride costly hoti hai", "neutral"),
("YouTube ke reviews hamesha ek jese nahi lagte, kabhi helpful kabhi waste of time", "neutral"),
("Easypaisa aur JazzCash dono apps daily kaam ke liye reliable hain, bas charges different hain", "neutral"),
("Daraz sales mein prices low hoti hain magar stock limited hota hai", "neutral"),
("Careem aur InDrive dono ka apna apna faida hai", "neutral"),
("FoodPanda discount codes kabhi apply ho jate hain aur kabhi expire milte hain", "neutral"),
("JazzCash ka interface average hai, simple bhi hai magar modern touch missing hai", "neutral"),
("Bykea ka location tracking kabhi accurate hota hai kabhi glitch", "neutral"),
("Easypaisa ka helpline kabhi helpful hota hai aur kabhi busy line milti hai", "neutral"),
("OLX par sellers ki honesty par depend karta hai experience", "neutral"),
("Daraz reviews se kabhi product sahi guess hota hai kabhi mislead", "neutral"),
("Careem captain kabhi friendly hota hai aur kabhi sirf normal drive deta hai", "neutral"),
("FoodPanda ki service har city mein ek jesi nahi hoti", "neutral"),
("YouTube vlogs kabhi entertaining lagte hain kabhi boring", "neutral"),
("JazzCash aur Easypaisa dono apps ek jesi lagti hain features ke hisaab se", "neutral"),
("Bykea ka fare reasonable lagta hai magar traffic delay common hai", "neutral"),
("Daraz pe quality kabhi achi hoti hai kabhi compromise lagti hai", "neutral"),
("Careem app theek chalti hai, sirf kabhi kabhi slow hoti hai", "neutral"),
("Easypaisa recharge process mostly smooth hai lekin kabhi error aa jata hai", "neutral"),
("OLX pe cheez milti bhi hai aur kabhi time waste bhi hota hai", "neutral"),
("FoodPanda app use karna easy hai, kuch options missing lagte hain", "neutral"),
("YouTube ke product reviews kabhi genuine lagte hain kabhi biased", "neutral"),
("JazzCash updates se kabhi app fast hoti hai kabhi aur slow ho jati hai", "neutral"),
("Daraz ka return policy thoda long hai magar chal jati hai", "neutral"),
("Careem aur Bykea dono rides try ki hain, dono ka experience mixed hai", "neutral"),
("FoodPanda rider kabhi fast pohanchta hai aur kabhi wait krwana padta hai", "neutral"),
("Easypaisa aur JazzCash ka transaction speed almost same lagta hai", "neutral"),
("OLX deals buyer aur seller dono par depend karti hain", "neutral"),
("Bykea se short rides theek hain, long rides comfortable nahi lagti", "neutral"),
("YouTube ka content har creator pe depend karta hai, quality vary karti hai", "neutral"),
("Daraz par delivery speed area ke hisaab se different hoti hai", "neutral"),
("Careem ka AC kabhi theek hota hai kabhi weak", "neutral"),
("FoodPanda ka UI simple hai lekin advanced options missing hain", "neutral")
]

df = pd.DataFrame(data, columns=["text", "sentiment"])
df.to_csv("urdu_roman_synthetic_data.csv", index=False, encoding="utf-8")

## Combining

In [93]:
# Concatenate the two DataFrames
combined_df = pd.concat([df, df_combined], ignore_index=True)

In [94]:
# shuffle the rows
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

# remove duplicates
combined_df.drop_duplicates(subset=['text'], inplace=True)

In [95]:
# Save the combined CSV
combined_df.to_csv("combined_sentiment_dataset.csv", index=False)

print("Combined dataset shape:", combined_df.shape)

Combined dataset shape: (3307, 2)


## Pure Urdu Synthetic Data

In [96]:
import pandas as pd

data = [
("جیز کیش سے بل جمع کروایا تو چند لمحوں میں ہو گیا، واقعی آسان سہولت ہے", "positive"),
("ایزی پیسہ کے ذریعے پیسے بھیجنا بہت سہل ہے، مجھے ہمیشہ اطمینان رہتا ہے", "positive"),
("فوڈ پانڈا پر کھانے کا آرڈر وقت پر پہنچا اور ذائقہ بھی بہت اچھا تھا", "positive"),
("دراز کی سیل سے میں نے کپڑے خریدے، معیار شاندار نکلا", "positive"),
("اولیکس پر پرانے موبائل اچھی قیمت پر بآسانی دستیاب ہو جاتے ہیں", "positive"),
("کریم کی سواری آرام دہ اور تیز رفتار تھی", "positive"),
("بائیکیا کے ذریعے سامان جلدی پہنچ گیا، ڈرائیور بھی خوش اخلاق تھا", "positive"),
("یوٹیوب ولاگ سے نئی جگہوں کے بارے میں دلچسپ معلومات ملیں", "positive"),
("یہ پروڈکٹ قیمت کے حساب سے بہترین ہے", "positive"),
("ایپ کا ڈیزائن صاف اور استعمال میں نہایت آسان ہے", "positive"),
("فوڈ پانڈا پر رعایتی کوڈ استعمال کر کے کافی بچت ہوئی", "positive"),
("جیز کیش سے موبائل ریچارج کرتے ہی بیلنس فوراً مل گیا", "positive"),
("دراز کی ڈیلیوری وقت پر پہنچی اور پیکنگ بھی عمدہ تھی", "positive"),
("بائیکیا کی بکنگ تیز اور بالکل آسان ہے", "positive"),
("اولیکس پر خریدار اور بیچنے والے دونوں جلدی رابطہ کر لیتے ہیں", "positive"),
("یوٹیوب ریویو دیکھ کر پروڈکٹ خریدنے کا فیصلہ آسان ہوا", "positive"),
("ایزی پیسہ سے اب بل دینا اور بھی سہل ہو گیا ہے", "positive"),
("کریم کے ڈسکاؤنٹ کوڈ نے سفر کی لاگت کم کر دی", "positive"),
("فوڈ پانڈا پر مقامی ریسٹورنٹس کی بڑی ورائٹی موجود ہے", "positive"),
("دراز پر خرید و فروخت کا عمل محفوظ اور مطمئن کن ہے", "positive"),
("یوٹیوب ولاگر نے معلومات کو دلچسپ انداز میں بیان کیا", "positive"),
("ایپ تیزی سے کھلتا ہے اور ہینگ نہیں کرتا", "positive"),
("اولیکس پر گاڑیوں کی خرید و فروخت آسانی سے ہو جاتی ہے", "positive"),
("بائیکیا کے کرائے عام صارف کے لیے مناسب ہیں", "positive"),
("کریم کی کسٹمر سپورٹ نے مسئلہ فوراً حل کر دیا", "positive"),
("دراز کی ریٹرن پالیسی قابل اعتماد اور سہل ہے", "positive"),
("فوڈ پانڈا پر ہر بار تازہ کھانا ملا ہے", "positive"),
("یوٹیوب ویڈیو کی کوالٹی اچھی اور واضح تھی", "positive"),
("ایزی پیسہ ایپ میں مسلسل نئی سہولتیں شامل ہو رہی ہیں", "positive"),
("جیز کیش پر لین دین ہر بار کامیاب رہا ہے", "positive"),
("بائیکیا کے ڈرائیور نے شائستگی سے بات کی", "positive"),
("کریم کی سواری نے سفر آسان اور خوشگوار بنا دیا", "positive"),
("یوٹیوب ریویو نے فیصلہ کرنے میں مدد دی", "positive"),
("اولیکس پر اشیاء بیچنا اور خریدنا دونوں سہولت سے ہوتے ہیں", "positive"),
("فوڈ پانڈا پر آرڈر دینے کا عمل آسان اور تیز ہے", "positive"),
("دراز کی قیمتیں اکثر دیگر جگہوں سے بہتر ہوتی ہیں", "positive"),
("ایپ میں نیویگیشن بالکل سیدھی اور سمجھنے میں آسان ہے", "positive"),
("یوٹیوب ولاگر نے سچائی سے تجربہ بیان کیا", "positive"),
("ایزی پیسہ نے روزمرہ زندگی کو سہل بنا دیا ہے", "positive"),
("جیز کیش پر ادائیگیوں کا ریکارڈ محفوظ اور قابل اعتماد ہے", "positive"),
("بائیکیا نے وقت پر پیکج پہنچایا", "positive"),
("کریم کا ڈرائیور خوش اخلاق اور مددگار تھا", "positive"),
("فوڈ پانڈا سے کھانے کی ترسیل ہمیشہ بروقت ملتی ہے", "positive"),
("دراز سے خریدا گیا پروڈکٹ معیار کے لحاظ سے بہترین تھا", "positive"),
("اولیکس پر خریدار فوراً رابطہ کرتے ہیں", "positive"),
("ایپ استعمال کرنا آسان اور تیز رفتار ہے", "positive"),
("یوٹیوب ویڈیو دیکھ کر کافی کچھ نیا سیکھنے کو ملا", "positive"),
("ایزی پیسہ کی سہولت گاؤں کے علاقوں میں بھی کارآمد ہے", "positive"),
("جیز کیش نے ادائیگیوں کا عمل بے حد سہل کر دیا ہے", "positive"),
("دراز کی کسٹمر سروس نے مسئلہ جلدی حل کیا", "positive"),
("جیز کیش سے پیسے بھیجنے کی کوشش کی مگر سسٹم بار بار فیل ہو رہا تھا", "negative"),
("ایزی پیسہ ایپ اکثر ہینگ ہو جاتی ہے، کافی وقت ضائع ہوتا ہے", "negative"),
("فوڈ پانڈا پر آرڈر کیا لیکن کھانا ٹھنڈا اور ذائقہ بھی خراب تھا", "negative"),
("دراز سے منگوایا ہوا سامان تصویر جیسا بالکل نہیں نکلا", "negative"),
("اولیکس پر جعلی اشتہارات بہت زیادہ ہیں، اعتماد نہیں رہتا", "negative"),
("کریم کے ڈرائیور نے غیر ضروری لمبا راستہ اختیار کیا", "negative"),
("بائیکیا کی سواری بار بار کینسل ہو جاتی ہے، پریشانی ہوتی ہے", "negative"),
("یوٹیوب ولاگر نے پروڈکٹ کی تعریف تو کی لیکن حقیقت مختلف نکلی", "negative"),
("ایپ کھلنے میں بہت وقت لیتا ہے، بار بار کریش کرتا ہے", "negative"),
("فوڈ پانڈا پر ڈسکاؤنٹ کوڈ لگانے کی کوشش کی مگر کام ہی نہیں کیا", "negative"),
("دراز کی ڈیلیوری کئی دن تاخیر سے پہنچی", "negative"),
("ایزی پیسہ سے بل جمع کرتے وقت بار بار ایرر آتا ہے", "negative"),
("جیز کیش کسٹمر سروس بالکل مددگار ثابت نہیں ہوئی", "negative"),
("بائیکیا کے ڈرائیور نے رویہ اچھا نہیں رکھا", "negative"),
("کریم کی سواری وقت پر نہیں آئی، کافی انتظار کرنا پڑا", "negative"),
("یوٹیوب ویڈیو کا مواد سطحی تھا اور کوئی فائدہ نہیں ہوا", "negative"),
("اولیکس پر دھوکہ دہی کے کیسز بہت سننے کو ملتے ہیں", "negative"),
("دراز پر ریٹرن پالیسی کا عمل بہت مشکل اور وقت لینے والا ہے", "negative"),
("فوڈ پانڈا پر مینو میں جو دکھایا گیا تھا وہ کھانے میں موجود نہیں تھا", "negative"),
("ایزی پیسہ میں رقم کٹ گئی لیکن وصول کنندہ کو نہیں ملی", "negative"),
("جیز کیش اکثر اپڈیٹ کے بعد مسائل پیدا کرتا ہے", "negative"),
("بائیکیا کا کرایہ پہلے سے زیادہ بڑھا دیا گیا ہے", "negative"),
("کریم کے ڈرائیور نے راستے میں ایئرکنڈیشن بھی نہیں چلایا", "negative"),
("یوٹیوب ولاگر نے پروڈکٹ کو بڑھا چڑھا کر بیان کیا", "negative"),
("دراز سے خریدے ہوئے جوتے کا سائز غلط بھیج دیا گیا", "negative"),
("اولیکس پر رابطہ کرنے والے زیادہ تر سنجیدہ نہیں ہوتے", "negative"),
("فوڈ پانڈا کی ڈیلیوری حد سے زیادہ لیٹ تھی", "negative"),
("ایزی پیسہ ایپ میں لاگ ان کرتے وقت بار بار مسئلہ آتا ہے", "negative"),
("جیز کیش پر رقم واپس لینے میں کئی دن لگ گئے", "negative"),
("بائیکیا کے ڈرائیور نے بکنگ کے بعد کال ہی نہیں اٹھائی", "negative"),
("کریم میں سفر کا کرایہ ایپ پر دکھائے گئے سے زیادہ لیا گیا", "negative"),
("یوٹیوب ویڈیو میں صرف اشتہار زیادہ تھا، معلومات کم تھیں", "negative"),
("دراز سے خریدا گیا سامان خراب حالت میں پہنچا", "negative"),
("اولیکس پر کچھ بیچنے والے جعلی پروفائل استعمال کرتے ہیں", "negative"),
("فوڈ پانڈا پر آرڈر ٹریکنگ صحیح کام نہیں کرتی", "negative"),
("ایزی پیسہ کا انٹرفیس پیچیدہ ہے، سمجھنا مشکل ہوتا ہے", "negative"),
("جیز کیش کی ایپ بعض اوقات بالکل کھلتی ہی نہیں", "negative"),
("بائیکیا کے سفر میں ڈرائیور نے بدتمیزی کی", "negative"),
("کریم کی کسٹمر سپورٹ نے مسئلہ حل کرنے میں وقت ضائع کیا", "negative"),
("یوٹیوب ولاگ میں سچائی کم اور ڈرامہ زیادہ تھا", "negative"),
("دراز پر جعلی ریویوز بہت زیادہ ہیں", "negative"),
("اولیکس پر محفوظ لین دین کی گارنٹی نہیں ہے", "negative"),
("فوڈ پانڈا پر پیکیجنگ بالکل غیر معیاری تھی", "negative"),
("ایزی پیسہ ایپ میں بار بار اپڈیٹ آتے ہیں جو مسائل بڑھا دیتے ہیں", "negative"),
("جیز کیش پر ریچارج کرنے کے بعد بیلنس تاخیر سے ملا", "negative"),
("بائیکیا کا سفر بار بار کینسل ہو کر وقت ضائع کرتا ہے", "negative"),
("کریم کی سواری کا جی پی ایس لوکیشن درست نہیں دکھا رہا تھا", "negative"),
("یوٹیوب ویڈیو کے تبصرے زیادہ کارآمد تھے بجائے ویڈیو کے", "negative"),
("دراز کے سیل میں اکثر چیزیں مہنگی ہی ملتی ہیں", "negative"),
("اولیکس پر خریدار سنجیدہ نہیں ہوتے، بس پوچھ کر غائب ہو جاتے ہیں", "negative"),
("جیز کیش کا استعمال عام ہے اور زیادہ تر لوگ بل اسی سے ادا کرتے ہیں", "neutral"),
("ایزی پیسہ کے ذریعے موبائل ریچارج کیا جا سکتا ہے", "neutral"),
("فوڈ پانڈا پر مختلف ریسٹورنٹس کا کھانا دستیاب ہے", "neutral"),
("دراز کی ایپ پر کپڑوں سے لے کر موبائل تک سب مل جاتا ہے", "neutral"),
("اولیکس پر پرانی چیزیں خریدنے اور بیچنے کا رجحان بڑھ رہا ہے", "neutral"),
("کریم شہر کے مختلف حصوں میں سروس فراہم کر رہا ہے", "neutral"),
("بائیکیا عموماً قریبی سفر کے لیے زیادہ استعمال ہوتی ہے", "neutral"),
("یوٹیوب پر کئی ولاگرز اپنے تجربات شیئر کرتے ہیں", "neutral"),
("یہ پروڈکٹ بازار میں بھی دستیاب ہے اور آن لائن بھی", "neutral"),
("ایپ میں مختلف فیچرز شامل ہیں جو روزمرہ میں مددگار ہیں", "neutral"),
("فوڈ پانڈا کے ذریعے کھانا گھر پر منگوایا جا سکتا ہے", "neutral"),
("جیز کیش کو زیادہ تر لوگ بل اور ریچارج کے لیے استعمال کرتے ہیں", "neutral"),
("ایزی پیسہ بینک ٹرانسفر کے لیے بھی استعمال ہوتا ہے", "neutral"),
("دراز پر بڑی سیل کے دوران رش زیادہ ہوتا ہے", "neutral"),
("یوٹیوب ریویو میں پروڈکٹ کی تفصیل بیان کی گئی تھی", "neutral"),
("اولیکس پر نئی اور پرانی دونوں قسم کی چیزیں مل جاتی ہیں", "neutral"),
("کریم کی بکنگ موبائل ایپ کے ذریعے کی جاتی ہے", "neutral"),
("بائیکیا زیادہ تر موٹر سائیکل سواری کے لیے استعمال ہوتی ہے", "neutral"),
("ایپ کا استعمال کچھ صارفین کے لیے آسان جبکہ کچھ کے لیے مشکل ہو سکتا ہے", "neutral"),
("فوڈ پانڈا کی ایپ میں کئی مقامی ریسٹورنٹس شامل ہیں", "neutral"),
("جیز کیش کی مقبولیت دن بدن بڑھ رہی ہے", "neutral"),
("ایزی پیسہ گاؤں کے علاقوں میں بھی دستیاب ہے", "neutral"),
("دراز پر الیکٹرانکس کی ورائٹی کافی بڑی ہے", "neutral"),
("یوٹیوب ولاگر نے سفر کے مناظر دکھائے تھے", "neutral"),
("اولیکس پر گاڑیوں کے اشتہارات بھی موجود ہیں", "neutral"),
("کریم کی سواری عموماً ایپ کے ذریعے ٹریک کی جا سکتی ہے", "neutral"),
("بائیکیا ایپ پر سفر کی تفصیل بکنگ کے بعد آ جاتی ہے", "neutral"),
("ایپ میں زبان کی تبدیلی کا آپشن بھی موجود ہے", "neutral"),
("فوڈ پانڈا کے مینو میں مختلف کچن کی ڈشز شامل ہیں", "neutral"),
("جیز کیش سے پیسے بھیجنے کا عمل تیز ہے مگر کبھی کبھی دیر ہو جاتی ہے", "neutral"),
("ایزی پیسہ کا استعمال زیادہ تر لوگ قریبی دکانوں سے بھی کرتے ہیں", "neutral"),
("دراز کی ایپ میں فلٹر کے ذریعے پروڈکٹ آسانی سے ڈھونڈا جا سکتا ہے", "neutral"),
("یوٹیوب پر مختلف پروڈکٹس کے ریویوز موجود ہیں", "neutral"),
("اولیکس کی ایپ کے ذریعے خریدار اور بیچنے والا براہ راست بات کر سکتے ہیں", "neutral"),
("کریم میں سفر کے اختتام پر ریٹنگ دینا لازمی ہے", "neutral"),
("بائیکیا پر سواری بک ہونے کے بعد ڈرائیور کی تفصیل بھیج دی جاتی ہے", "neutral"),
("ایپ کا انٹرفیس سادہ رکھا گیا ہے تاکہ صارفین آسانی سے استعمال کر سکیں", "neutral"),
("فوڈ پانڈا پر ڈیلیوری چارجز علاقے کے حساب سے مختلف ہوتے ہیں", "neutral"),
("جیز کیش موبائل نیٹ ورک کے صارفین کے لیے زیادہ آسانی پیدا کرتا ہے", "neutral"),
("ایزی پیسہ ایپ میں حالیہ لین دین کی تفصیل دیکھی جا سکتی ہے", "neutral"),
("دراز پر پروڈکٹ کی تصاویر بھی ساتھ دی جاتی ہیں", "neutral"),
("یوٹیوب پر ہر روز نئے ولاگ اپلوڈ ہوتے ہیں", "neutral"),
("اولیکس پر گھریلو سامان کے اشتہارات بھی بڑی تعداد میں ہیں", "neutral"),
("کریم کی سروس دن رات دستیاب ہے", "neutral"),
("بائیکیا کا استعمال زیادہ تر نوجوان کرتے ہیں", "neutral"),
("ایپ میں نوٹیفکیشن کے ذریعے نئے فیچرز کا پتا چلتا ہے", "neutral"),
("فوڈ پانڈا پر مقامی اور بین الاقوامی دونوں برانڈز کا کھانا ملتا ہے", "neutral"),
("جیز کیش سے ادائیگی موبائل ایپ اور دکان دونوں پر کی جا سکتی ہے", "neutral"),
("ایزی پیسہ بینک اکاؤنٹ کے بغیر بھی استعمال کیا جا سکتا ہے", "neutral"),
("جیز کیش کے ذریعے بل کی ادائیگی نے زندگی آسان بنا دی ہے", "positive"),
("ایزی پیسہ کی ایپ استعمال کرنے میں بالکل مشکل پیش نہیں آئی", "positive"),
("بائیکا کی سواری وقت پر پہنچی اور سفر آرام دہ رہا", "positive"),
("کریم ڈرائیور کا رویہ بہت خوش اخلاق تھا", "positive"),
("فوڈ پانڈا سے کھانے کی ڈلیوری تازہ اور جلد ملی", "positive"),
("دراز کی سیل میں اصل رعایتیں ملیں، قیمتیں واقعی کم تھیں", "positive"),
("او ایل ایکس پر خریدا گیا موبائل بالکل ٹھیک نکلا", "positive"),
("یوٹیوب ویڈیو سے نئی ترکیبیں سیکھنے کو ملیں", "positive"),
("ایزی پیسہ کا کیش بیک آفر بہت فائدہ مند ثابت ہوا", "positive"),
("جیز کیش ایپ کے نوٹیفکیشن وقت پر معلومات فراہم کرتے ہیں", "positive"),
("دراز سے لیپ ٹاپ کا آرڈر وقت پر اور محفوظ پیکنگ کے ساتھ ملا", "positive"),
("فوڈ پانڈا پر مقامی ریستوران کا ذائقہ شاندار تھا", "positive"),
("کریم ایپ کی جی پی ایس ٹریکنگ درست کام کر رہی تھی", "positive"),
("یوٹیوب ویڈیو کے تبصرے بھی معلوماتی تھے", "positive"),
("ایزی پیسہ سے گاؤں میں بھی رقم بھیجنا ممکن ہوا", "positive"),
("جیز کیش کی صارف خدمت فوری جواب دیتی ہے", "positive"),
("بائیکا کے کرائے مناسب اور جیب پر ہلکے ہیں", "positive"),
("کریم ایپ میں سواری بک کرنا آسان ہے", "positive"),
("فوڈ پانڈا کے ڈسکاونٹ کوڈ نے اچھا خاصا پیسہ بچایا", "positive"),
("دراز پر کپڑوں کے ڈیزائن حقیقت میں بھی اتنے ہی اچھے تھے", "positive"),
("یوٹیوب چینل کی ویڈیوز نے میرا علم بڑھایا", "positive"),
("ایزی پیسہ کے ذریعے اسکول فیس جمع کروانا سہل ہوا", "positive"),
("جیز کیش نے بینک جانے کی جھنجھٹ ختم کر دی", "positive"),
("کریم ڈرائیور نے راستہ بدل کر ٹریفک سے بچایا", "positive"),
("فوڈ پانڈا کی رائیڈر نے آرڈر مسکراتے ہوئے دیا", "positive"),
("دراز کی کسٹمر سروس نے مسئلہ فوراً حل کر دیا", "positive"),
("او ایل ایکس پر بک کیا گیا فریج وقت پر مل گیا", "positive"),
("یوٹیوب وڈیو نے بہترین ہدایات فراہم کیں", "positive"),
("ایزی پیسہ ایپ کا نیا ڈیزائن بہت پسند آیا", "positive"),
("جیز کیش کے ڈپازٹ کا عمل تیز رفتار ہے", "positive"),
("بائیکا ڈرائیور خوش اخلاق اور مددگار تھا", "positive"),
("کریم کی پروموشن آفر واقعی فائدہ مند رہی", "positive"),
("فوڈ پانڈا پر میٹھے کے آرڈر نے دل خوش کر دیا", "positive"),
("دراز پر آن لائن خریداری کا تجربہ شاندار رہا", "positive"),
("یوٹیوب چینل نے نئی زبان سیکھنے میں مدد کی", "positive"),
("ایزی پیسہ کی ایپ ہر اپ ڈیٹ کے ساتھ بہتر ہوتی جا رہی ہے", "positive"),
("جیز کیش کے ذریعے ایزی لوڈ آسانی سے ہو گیا", "positive"),
("کریم سواری نے وقت بچایا اور سفر خوشگوار بنایا", "positive"),
("فوڈ پانڈا کی ایپ میں آرڈر ٹریکنگ مفید ہے", "positive"),
("دراز پر ادائیگی کے طریقے آسان اور محفوظ ہیں", "positive"),
("او ایل ایکس پر پرانی چیزیں بھی اچھی حالت میں ملتی ہیں", "positive"),
("یوٹیوب پر ویڈیوز دیکھ کر نئے آئیڈیاز ملتے ہیں", "positive"),
("ایزی پیسہ کی سہولت دیہات میں بھی پہنچ چکی ہے", "positive"),
("جیز کیش سے رقم کی ترسیل بہت آسان ہے", "positive"),
("بائیکا کی بائیک صاف ستھری اور محفوظ تھی", "positive"),
("کریم ایپ نے سفر کی پریشانی کم کر دی ہے", "positive"),
("فوڈ پانڈا پر نئے ریستوران آزمانے کا موقع ملا", "positive"),
("دراز کی ڈیلیوری وقت پر اور بہترین پیکنگ میں تھی", "positive"),
("یوٹیوب وڈیو کی کوالٹی اعلیٰ تھی", "positive"),
("ایزی پیسہ کی ایپ قابل اعتماد ہے", "positive"),
("جیز کیش ایپ بار بار کریش ہو جاتی ہے", "negative"),
("ایزی پیسہ کے سرور ہمیشہ مصروف رہتے ہیں", "negative"),
("بائیکا ڈرائیور نے لوکیشن ڈھونڈنے میں بہت وقت ضائع کیا", "negative"),
("کریم کی سواری دیر سے پہنچی", "negative"),
("فوڈ پانڈا پر کھانے کی مقدار کم تھی", "negative"),
("دراز کی سیل میں قیمتیں ویسی ہی تھیں، رعایت نہیں ملی", "negative"),
("او ایل ایکس پر بیچا گیا موبائل خراب نکلا", "negative"),
("یوٹیوب وڈیو میں معلومات نامکمل تھیں", "negative"),
("ایزی پیسہ ایپ کا انٹرفیس پیچیدہ ہے", "negative"),
("جیز کیش کا پاس ورڈ ری سیٹ کرنا مشکل ہے", "negative"),
("کریم ایپ میں جی پی ایس صحیح جگہ نہیں دکھاتا", "negative"),
("فوڈ پانڈا ڈلیوری بوائے بدتمیز تھا", "negative"),
("دراز سے آرڈر کیا کپڑا معیار میں کمزور نکلا", "negative"),
("او ایل ایکس پر دھوکہ دہی کے واقعات بڑھ گئے ہیں", "negative"),
("یوٹیوب وڈیو بار بار بفر ہوتی رہی", "negative"),
("ایزی پیسہ کا کیش بیک وعدے کے مطابق نہیں ملا", "negative"),
("جیز کیش پر رقم بھیجنے میں بہت دیر لگتی ہے", "negative"),
("بائیکا کی بائیک پرانی اور غیر آرام دہ تھی", "negative"),
("کریم ڈرائیور نے غلط راستہ اختیار کیا", "negative"),
("فوڈ پانڈا کا کھانا ٹھنڈا پہنچا", "negative"),
("دراز پر واپسی کا عمل پیچیدہ ہے", "negative"),
("او ایل ایکس پر دی گئی اشتہار کی معلومات جھوٹی تھیں", "negative"),
("یوٹیوب پر وڈیو کا معیار خراب تھا", "negative"),
("ایزی پیسہ ایپ بار بار اپ ڈیٹ مانگتی ہے", "negative"),
("جیز کیش پر فیس زیادہ کاٹی جاتی ہے", "negative"),
("بائیکا ڈرائیور وقت پر نہیں آیا", "negative"),
("کریم کی ایپ میں بکنگ کا مسئلہ تھا", "negative"),
("فوڈ پانڈا پر ریستوران کا مینو درست نہیں تھا", "negative"),
("دراز پر ڈیلیوری وقت سے زیادہ لیٹ تھی", "negative"),
("او ایل ایکس پر خریدا گیا پنکھا خراب نکلا", "negative"),
("یوٹیوب وڈیو میں آواز کی کوالٹی ناقص تھی", "negative"),
("ایزی پیسہ کے ایجنٹ نے صحیح رہنمائی نہیں کی", "negative"),
("جیز کیش کا نیٹ ورک اکثر ڈاؤن رہتا ہے", "negative"),
("بائیکا کی رائیڈ بہت مہنگی تھی", "negative"),
("کریم ڈرائیور نے ایئرکنڈیشن بند رکھا", "negative"),
("فوڈ پانڈا پر کھانے کی قیمتیں غیر مناسب ہیں", "negative"),
("دراز سے خریدا گیا بیگ خراب حالت میں پہنچا", "negative"),
("او ایل ایکس پر فراڈ کرنے والے لوگ زیادہ ہیں", "negative"),
("یوٹیوب چینل پر غیر ضروری اشتہارات بہت تھے", "negative"),
("ایزی پیسہ کا نظام غیر مستحکم ہے", "negative"),
("جیز کیش ایپ میں لاگ ان کا مسئلہ ہے", "negative"),
("بائیکا ڈرائیور نے اضافی کرایہ مانگا", "negative"),
("کریم سواری کے دوران ڈرائیور موبائل پر مصروف رہا", "negative"),
("فوڈ پانڈا پر کھانے کی کوالٹی کمزور ہے", "negative"),
("دراز پر خریدی گئی کتاب پھٹی ہوئی نکلی", "negative"),
("او ایل ایکس پر رابطہ نمبر جعلی تھے", "negative"),
("یوٹیوب وڈیو غیر معیاری تھی", "negative"),
("ایزی پیسہ کا کسٹمر سپورٹ جواب نہیں دیتا", "negative"),
("جیز کیش ایپ کے فیچرز عام معیار کے ہیں", "neutral"),
("ایزی پیسہ سے کبھی فائدہ ملتا ہے کبھی نہیں", "neutral"),
("بائیکا کی سواری بعض اوقات وقت پر ملتی ہے", "neutral"),
("کریم کا کرایہ کبھی زیادہ کبھی کم ہوتا ہے", "neutral"),
("فوڈ پانڈا پر کھانا کبھی اچھا کبھی درمیانہ ہوتا ہے", "neutral"),
("دراز پر کچھ چیزیں مناسب قیمت پر اور کچھ مہنگی ہیں", "neutral"),
("او ایل ایکس پر اچھے اور خراب دونوں قسم کے سودے ہوتے ہیں", "neutral"),
("یوٹیوب ویڈیوز میں کچھ معلوماتی اور کچھ غیر اہم ہوتی ہیں", "neutral"),
("ایزی پیسہ ایپ کبھی تیز چلتی ہے اور کبھی رک جاتی ہے", "neutral"),
("جیز کیش پر بعض اوقات نیٹ ورک کا مسئلہ ہوتا ہے", "neutral"),
("کریم کی سروس کبھی اچھی کبھی مایوس کن ہوتی ہے", "neutral"),
("فوڈ پانڈا کا کھانا کبھی دیر سے اور کبھی وقت پر آتا ہے", "neutral"),
("دراز کی ڈیلیوری بعض اوقات جلدی ہو جاتی ہے", "neutral"),
("او ایل ایکس پر بیچے جانے والے سامان کی کوالٹی مختلف رہتی ہے", "neutral"),
("یوٹیوب چینل کی ویڈیوز میں معیار مختلف ہوتا ہے", "neutral"),
("ایزی پیسہ کا کیش بیک ہر بار نہیں ملتا", "neutral"),
("جیز کیش ایپ کبھی اپ ڈیٹ کے بعد بہتر اور کبھی بدتر لگتی ہے", "neutral"),
("بائیکا ڈرائیور بعض اوقات راستہ صحیح جانتے ہیں", "neutral"),
("کریم کی ایپ کبھی سست چلتی ہے", "neutral"),
("فوڈ پانڈا پر ریستوران کے ریویو ہمیشہ درست نہیں ہوتے", "neutral"),
("دراز کی کسٹمر سروس کبھی مددگار اور کبھی نہیں", "neutral"),
("او ایل ایکس پر کچھ لوگ ایماندار اور کچھ نہیں ہوتے", "neutral"),
("یوٹیوب پر کچھ وڈیوز وقت ضائع کرتی ہیں", "neutral"),
("ایزی پیسہ کی سروس بعض اوقات اچھی اور بعض اوقات ناقص رہتی ہے", "neutral"),
("جیز کیش پر ادائیگی کبھی فوراً اور کبھی دیر سے ہوتی ہے", "neutral"),
("بائیکا سواری کا معیار ہمیشہ ایک جیسا نہیں رہتا", "neutral"),
("کریم کی سہولت کبھی آرام دہ کبھی تھکا دینے والی ہوتی ہے", "neutral"),
("فوڈ پانڈا کے آرڈر کبھی درست کبھی غلط آتے ہیں", "neutral"),
("دراز پر کچھ خریدار مطمئن اور کچھ نالاں ہوتے ہیں", "neutral"),
("او ایل ایکس کے اشتہارات کبھی حقیقی اور کبھی جعلی نکلتے ہیں", "neutral"),
("یوٹیوب چینل کی معلومات کبھی کارآمد کبھی غیر ضروری ہوتی ہیں", "neutral"),
("ایزی پیسہ ایپ کا استعمال بعض کے لیے آسان بعض کے لیے مشکل ہے", "neutral"),
("جیز کیش پر کبھی فیس زیادہ اور کبھی کم لگتی ہے", "neutral"),
("بائیکا کی سواری کبھی سستی کبھی مہنگی لگتی ہے", "neutral"),
("کریم ڈرائیور بعض اوقات رویہ اچھا اور بعض اوقات نہیں رکھتے", "neutral"),
("فوڈ پانڈا کا کھانا کبھی ذائقے دار کبھی معمولی ہوتا ہے", "neutral"),
("دراز پر ملنے والے پروڈکٹس کبھی اعلیٰ کبھی کم معیار کے ہوتے ہیں", "neutral"),
("او ایل ایکس پر خریدوفروخت کا تجربہ کبھی مثبت کبھی منفی ہوتا ہے", "neutral"),
("یوٹیوب پر تبصرے کبھی مددگار کبھی فضول لگتے ہیں", "neutral"),
("ایزی پیسہ کے نوٹیفکیشن کبھی وقت پر آتے ہیں کبھی نہیں", "neutral"),
("جیز کیش ایپ کا ڈیزائن کبھی پسند آتا ہے کبھی نہیں", "neutral"),
("بائیکا کی بائیک کبھی آرام دہ کبھی نہیں ہوتی", "neutral"),
("کریم سواری کبھی بروقت اور کبھی تاخیر سے پہنچتی ہے", "neutral"),
("فوڈ پانڈا پر پیکنگ کبھی اچھی کبھی خراب ہوتی ہے", "neutral"),
("دراز کی سروس کبھی متاثر کن اور کبھی عام سی ہوتی ہے", "neutral"),
("او ایل ایکس کے سودے کبھی فائدہ مند اور کبھی نقصان دہ ہوتے ہیں", "neutral"),
("یوٹیوب ویڈیو کا معیار کبھی بہتر کبھی کمزور ہوتا ہے", "neutral")
]

df = pd.DataFrame(data, columns=["text", "sentiment"])
df.to_csv("urdu_synthetic_data.csv", index=False, encoding="utf-8")

## Final File

In [ ]:
# Concatenate the two DataFrames
combined_df = pd.concat([combined_df, df], ignore_index=True)

# shuffle the rows
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

# remove duplicates
combined_df.drop_duplicates(subset=['text'], inplace=True)

# Save the combined CSV
combined_df.to_csv("combined_sentiment_dataset.csv", index=False)

print("Combined dataset shape:", combined_df.shape)

Combined dataset shape: (3601, 2)
